In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import itertools
import pprint as pp
from sklearn.linear_model import LinearRegression
from tqdm import trange
from sklearn.linear_model import LogisticRegression

__Задача 1__

Используя данные о хоккеистах (скачать можно [здесь](https://habr.com/ru/post/301340/)), проверьте, является ли среди хоккеистов из Финляндии, Норвегии и Дании значимым отличие роста. В качестве ответа запишите 1, если является, и 0 иначе.

In [18]:
#загрузили данные
df = pd.read_csv('hockey_players.csv')
df.head()

,year,country,no,name,position,side,height,weight,birth,club,age,cohort,bmi
0,2001,RUS,10,tverdovsky oleg,D,L,185,84.0,1976-05-18,anaheim mighty ducks,24.952772,1976,24.543462
1,2001,RUS,2,vichnevsky vitali,D,L,188,86.0,1980-03-18,anaheim mighty ducks,21.119781,1980,24.332277
2,2001,RUS,26,petrochinin evgeni,D,L,182,95.0,1976-02-07,severstal cherepovetal,25.229295,1976,28.680111
3,2001,RUS,28,zhdan alexander,D,R,178,85.0,1971-08-28,ak bars kazan,29.675565,1971,26.827421
4,2001,RUS,32,orekhovsky oleg,D,R,175,88.0,1977-11-03,dynamo moscow,23.490760,1977,28.734694


In [19]:
#быстрая проверка
df.describe()

,year,no,height,weight,age,cohort,bmi
count,6292.000000,6292.000000,6292.000000,6292.000000,6292.000000,6292.000000,6292.000000
mean,2008.556421,29.645105,183.811507,87.621662,27.280556,1981.154800,25.917179
std,4.593766,23.474342,5.383858,6.963876,4.276149,6.314963,1.478541
min,2001.000000,1.000000,165.000000,65.000000,17.642710,1963.000000,19.460881
25%,2005.000000,12.000000,180.000000,83.000000,24.136893,1976.000000,24.967113
50%,2009.000000,24.000000,183.000000,87.000000,27.013005,1981.000000,25.925926
75%,2013.000000,39.000000,188.000000,92.000000,30.123888,1986.000000,26.874496
max,2016.000000,98.000000,205.000000,117.000000,43.206023,1998.000000,30.930438


In [20]:
#убрали дубли, оставили только последнюю страну за которую играет хоккеист
df_dedup = df.drop_duplicates(subset=['name', 'birth'], keep='last', inplace=False, ignore_index=False)

In [21]:
df_dedup.describe()

,year,no,height,weight,age,cohort,bmi
count,3294.000000,3294.000000,3294.000000,3294.000000,3294.000000,3294.000000,3294.000000
mean,2008.329083,29.225562,183.894353,87.937614,27.847381,1980.365513,25.986522
std,5.092025,23.246428,5.476317,7.036507,4.398137,6.802670,1.481433
min,2001.000000,1.000000,165.000000,65.000000,17.686516,1963.000000,19.460881
25%,2004.000000,12.000000,180.000000,83.000000,24.672142,1975.000000,25.057360
50%,2007.000000,23.000000,184.000000,88.000000,27.603012,1980.000000,25.978680
75%,2013.000000,38.000000,188.000000,93.000000,30.819986,1985.000000,26.880935
max,2016.000000,97.000000,205.000000,117.000000,43.206023,1998.000000,30.930438


In [22]:
df_dedup.head()

,year,country,no,name,position,side,height,weight,birth,club,age,cohort,bmi
0,2001,RUS,10,tverdovsky oleg,D,L,185,84.0,1976-05-18,anaheim mighty ducks,24.952772,1976,24.543462
1,2001,RUS,2,vichnevsky vitali,D,L,188,86.0,1980-03-18,anaheim mighty ducks,21.119781,1980,24.332277
2,2001,RUS,26,petrochinin evgeni,D,L,182,95.0,1976-02-07,severstal cherepovetal,25.229295,1976,28.680111
4,2001,RUS,32,orekhovsky oleg,D,R,175,88.0,1977-11-03,dynamo moscow,23.490760,1977,28.734694
6,2001,RUS,6,evstafiev andrei,D,L,176,84.0,1972-08-09,lokomotiv yaroslavl,28.725530,1972,27.117769


In [26]:
#выяснили как закодированы страны
print(df_dedup.drop_duplicates(subset='country')['country'])

0       RUS
28      AUT
47      BLR
70      CAN
99      CZE
123     FIN
149     GER
171     ITA
193     JPN
216     LAT
237     NOR
263     SUI
283     SVK
306     SWE
331     UKR
356     USA
590     POL
614     SLO
878     DEN
1292    FRA
1369    KAZ
3342    HUN
Name: country, dtype: object


In [42]:
#собрали росты в три отдельные выборки
fin = df_dedup.loc[df_dedup['country'] == 'FIN']['height'].values
nor = df_dedup.loc[df_dedup['country'] == 'NOR']['height'].values
den = df_dedup.loc[df_dedup['country'] == 'DEN']['height'].values

In [43]:
#размеры выборок
fin_n = fin.shape[0]
nor_n = nor.shape[0]
den_n = den.shape[0]

In [53]:
#средние по выборке
fin_mean = fin.mean()
nor_mean = nor.mean()
den_mean = den.mean()

fin_mean, nor_mean, den_mean

(184.4278606965174, 182.80612244897958, 184.37313432835822)

In [46]:
#собрали общую выборку и получили ее среднее
y = np.concatenate([fin, nor, den])
y_mean = y.mean()

In [47]:
#получили значения ssb & ssw
SS_b = fin_n * (fin_mean - y_mean) ** 2 + nor_n * (nor_mean - y_mean) ** 2 + den_n * (den_mean - y_mean) ** 2

SS_w = ((fin - fin_mean) ** 2).sum() + ((nor - nor_mean) ** 2).sum() + ((den - den_mean) ** 2).sum()

SS_b, SS_w

(194.30269154299393, 10985.863590212204)

In [50]:
#получили оценку дисперсии
k = 3
n = fin_n + nor_n + den_n

k1 = k - 1
k2 = n - k

sigma2_b = SS_b / k1
sigma2_w = SS_w / k2

sigma2_b, sigma2_w

(97.15134577149696, 25.548519977237685)

In [51]:
#получили f-статистику
F = sigma2_b / sigma2_w
F

3.8026212813131024

In [52]:
#нашли критическую область
alpha = 0.05

t = stats.f.ppf(1 - alpha, k1, k2)
t

3.016700272452327

In [54]:
#сравнили с scipy
stats.f_oneway(fin, nor, den)

F_onewayResult(statistic=3.8026212813130664, pvalue=0.023066247867524335)

__Ответ:__

Низкое значение p-value подтверждает, что 0 гипотеза (отличие среднего роста хоккейстов значимо) - верно, это так же подтверждается тем, что статистика попала в критическую область 3.80 > 3.01